In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()
data = spark.read.csv('dbfs:/FileStore/shared_uploads/gkantirisrafael@gmail.com/SMSSpamCollection',inferSchema=True,sep='\t')

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import length
data = data.withColumn('length',length(data['text']))

In [0]:
#We can see that the average length of the spam sms is longer than the normal sms
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham| 71.4545266210897|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
#tf = Term frequency, idf = Inverse document frequency
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data)

In [0]:
clean_data = cleaner.transform(data)

In [0]:
clean_data = clean_data.select('label','features')

In [0]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [0]:
training,test = clean_data.randomSplit([0.7,0.3])

In [0]:
my_spam_detector = nb.fit(training)

In [0]:
test_results = my_spam_detector.transform(test)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-790.85149057146...|[1.0,8.4048323566...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-591.24452123209...|[0.99999999986063...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-801.99960801431...|[1.0,4.8977762663...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1208.4974551588...|[1.0,8.6041211030...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-664.29632549834...|[1.0,2.2390633535...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1362.6882939337...|[1.0,7.7179293676...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-990.27721801450...|[0.99999977676425...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-992.03256125345...|[1.0,5.0335562752...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1282.7916144709...|[1.0,2.6222359986...|       0.0|
|  0.0|(13424,[0

In [0]:
from pyspark.ml.evaluation  import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator()

In [0]:
my_acc = acc_eval.evaluate(test_results)

In [0]:
print('Accuracy of NB model:',my_acc)

Accuracy of NB model: 0.9251594416129116
